In [121]:
import argparse
import json
import os
import re
import time
import uuid
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from lxml import etree

In [122]:
_BASE_URL = 'http://papers.nips.cc'
_CRAWLING_WAIT_TIME = 0.3
_NEURIPS_NAMESPACE = uuid.UUID('5ee6531f-0d79-4cf1-8da6-dc83cb553336')
_FIRST_YEAR = 1988
_PDF_FOLDER = 'pdfs'
_OUTPUT_PAPERS_FILE = 'papers_data.json'

In [123]:
class NipsPaper:
    def __init__(self,year,title,doi,url,authors,abstract):
        self.year = year
        self.title = title
        self.doi = doi
        self.abstract = abstract
        self.authors = authors
        self.url = url
    
    
    def to_json(self):
        return {
            'id': self.doi,
            'title': self.title,
            'year' : self.year,
            'url': self.url,
            'abstract': self.abstract,
            'authors': self.authors
        }

In [124]:
def get_conference_links(year_from, year_to):
    base = _BASE_URL + '/paper_files/paper/'
    number_year_from = year_from - _FIRST_YEAR + 1
    number_year_to = year_to - _FIRST_YEAR + 1
    for idx, i in enumerate(range(number_year_from, number_year_to + 1)):
        year = str(year_from + idx)
        url = base + year
        yield (url,year)

        
def get_papers_year(url):
    base = _BASE_URL
    url_request = requests.get(url)
    soup = BeautifulSoup(url_request.content,'html5lib')
    papers_in_html = soup.find_all('a',attrs = {'title':'paper title'})
    for paper in papers_in_html:
        paper_link = base + paper["href"]
        yield paper_link

        
def get_paper_info(paper_url):
    url_request = requests.get(paper_url)
    soup = BeautifulSoup(url_request.content,'html5lib')
    #Paper title
    paper_title = soup.find('title').text
    #Paper authors
    authors_tags = soup.find_all('meta',attrs = {'name':'citation_author'})
    authors = []
    for author in authors_tags:
        authors.append(author['content'])
    #Paper publication date
    date = soup.find('meta',attrs = {'name':'citation_publication_date'})['content']
    #Paper url
    article_url = soup.find('meta',attrs = {'name':'citation_pdf_url'})['content']
    #Abstract 
    abstract_tag = soup.find('h4',text='Abstract')
    abstract_text = ''
    for p in abstract_tag.find_all_next('p'):
        if len(p.text) != 0:
            abstract_text += p.text
            break
    #Building paper ID
    splitting_url = article_url.split('/')
    index_hash = splitting_url.index("file")
    paper_id = splitting_url[index_hash + 1].split('-')[0]
    return NipsPaper(date,paper_title,paper_id,article_url,authors,abstract_text)

In [88]:
dumping_json = []
start_year = 2012
end_year = 2014
cpt = 0
base_folder = 'NIPS_papers_'
for url in tqdm(get_conference_links(start_year,end_year),'getting_conference_links'):
    time.sleep(_CRAWLING_WAIT_TIME)
    output_folder = base_folder + str(start_year + cpt)
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
    for paper_link in tqdm(get_papers_year(url[0])):
        time.sleep(_CRAWLING_WAIT_TIME)
        nips_paper = get_paper_info(paper_link)
        dumping_json.append(nips_paper.to_json())
    cpt += 1
with open('dataset.json', 'w') as file:
    json.dump(dumping_json, file, indent=2)

getting_conference_links: 0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:03,  3.28s/it]
2it [00:04,  2.22s/it]
3it [00:06,  1.89s/it]
4it [00:07,  1.73s/it]
5it [00:09,  1.64s/it]
6it [00:10,  1.55s/it]
7it [00:12,  1.51s/it]
8it [00:13,  1.48s/it]
9it [00:14,  1.50s/it]
10it [00:16,  1.52s/it]
11it [00:18,  1.52s/it]
12it [00:19,  1.60s/it]
13it [00:21,  1.63s/it]
14it [00:23,  1.67s/it]
15it [00:24,  1.62s/it]
16it [00:26,  1.59s/it]
17it [00:27,  1.56s/it]
18it [00:29,  1.55s/it]
19it [00:30,  1.51s/it]
20it [00:32,  1.52s/it]
21it [00:33,  1.52s/it]
22it [00:35,  1.49s/it]
23it [00:36,  1.49s/it]
24it [00:38,  1.52s/it]
25it [00:39,  1.51s/it]
26it [00:41,  1.50s/it]
27it [00:42,  1.53s/it]
28it [00:44,  1.55s/it]
29it [00:46,  1.55s/it]
30it [00:47,  1.52s/it]
31it [00:48,  1.49s/it]
32it [00:50,  1.47s/it]
33it [00:51,  1.46s/it]
34it [00:53,  1.45s/it]
35it [00:54,  1.45s/it]
36it [00:56,  1.45s/it]
37it [00:57,  1.43s/it]
38it [00:58,  1.44s/it]
39it [01:00,  1.44s/it]
40it [01:

295it [07:12,  1.39s/it]
296it [07:14,  1.42s/it]
297it [07:15,  1.41s/it]
298it [07:17,  1.40s/it]
299it [07:18,  1.39s/it]
300it [07:19,  1.38s/it]
301it [07:21,  1.39s/it]
302it [07:22,  1.39s/it]
303it [07:24,  1.40s/it]
304it [07:25,  1.41s/it]
305it [07:26,  1.41s/it]
306it [07:28,  1.41s/it]
307it [07:29,  1.43s/it]
308it [07:31,  1.42s/it]
309it [07:32,  1.41s/it]
310it [07:34,  1.41s/it]
311it [07:35,  1.40s/it]
312it [07:36,  1.39s/it]
313it [07:38,  1.38s/it]
314it [07:39,  1.37s/it]
315it [07:40,  1.37s/it]
316it [07:42,  1.41s/it]
317it [07:43,  1.42s/it]
318it [07:45,  1.44s/it]
319it [07:46,  1.47s/it]
320it [07:48,  1.47s/it]
321it [07:49,  1.43s/it]
322it [07:50,  1.42s/it]
323it [07:52,  1.40s/it]
324it [07:53,  1.40s/it]
325it [07:55,  1.41s/it]
326it [07:56,  1.42s/it]
327it [07:58,  1.41s/it]
328it [07:59,  1.42s/it]
329it [08:00,  1.41s/it]
330it [08:02,  1.42s/it]
331it [08:03,  1.43s/it]
332it [08:05,  1.43s/it]
333it [08:06,  1.46s/it]
334it [08:08,  1.44s/it]


218it [05:15,  1.45s/it]
219it [05:16,  1.45s/it]
220it [05:18,  1.45s/it]
221it [05:20,  1.74s/it]
222it [05:22,  1.72s/it]
223it [05:23,  1.68s/it]
224it [05:25,  1.59s/it]
225it [05:26,  1.52s/it]
226it [05:27,  1.47s/it]
227it [05:29,  1.46s/it]
228it [05:30,  1.46s/it]
229it [05:32,  1.44s/it]
230it [05:33,  1.42s/it]
231it [05:34,  1.40s/it]
232it [05:36,  1.42s/it]
233it [05:37,  1.44s/it]
234it [05:39,  1.44s/it]
235it [05:40,  1.42s/it]
236it [05:41,  1.40s/it]
237it [05:43,  1.41s/it]
238it [05:44,  1.41s/it]
239it [05:46,  1.45s/it]
240it [05:47,  1.45s/it]
241it [05:49,  1.48s/it]
242it [05:50,  1.49s/it]
243it [05:52,  1.46s/it]
244it [05:53,  1.45s/it]
245it [05:55,  1.43s/it]
246it [05:56,  1.45s/it]
247it [05:57,  1.44s/it]
248it [05:59,  1.46s/it]
249it [06:00,  1.43s/it]
250it [06:02,  1.45s/it]
251it [06:07,  2.42s/it]
252it [06:08,  2.22s/it]
253it [06:10,  2.14s/it]
254it [06:12,  1.90s/it]
255it [06:13,  1.76s/it]
256it [06:14,  1.67s/it]
257it [06:16,  1.60s/it]


151it [03:59,  1.56s/it]
152it [04:01,  1.51s/it]
153it [04:03,  1.83s/it]
154it [04:05,  1.75s/it]
155it [04:06,  1.66s/it]
156it [04:08,  1.64s/it]
157it [04:09,  1.56s/it]
158it [04:11,  1.52s/it]
159it [04:12,  1.52s/it]
160it [04:14,  1.54s/it]
161it [04:16,  1.77s/it]
162it [04:20,  2.32s/it]
163it [04:21,  2.09s/it]
164it [04:23,  1.94s/it]
165it [04:24,  1.81s/it]
166it [04:26,  1.71s/it]
167it [04:27,  1.64s/it]
168it [04:29,  1.62s/it]
169it [04:30,  1.59s/it]
170it [04:32,  1.56s/it]
171it [04:33,  1.55s/it]
172it [04:35,  1.51s/it]
173it [04:36,  1.54s/it]
174it [04:38,  1.54s/it]
175it [04:39,  1.51s/it]
176it [04:41,  1.47s/it]
177it [04:42,  1.45s/it]
178it [04:43,  1.44s/it]
179it [04:45,  1.42s/it]
180it [04:46,  1.43s/it]
181it [04:48,  1.43s/it]
182it [04:49,  1.43s/it]
183it [04:51,  1.42s/it]
184it [04:52,  1.42s/it]
185it [04:53,  1.44s/it]
186it [04:55,  1.43s/it]
187it [04:56,  1.42s/it]
188it [04:58,  1.45s/it]
189it [04:59,  1.42s/it]
190it [05:01,  1.43s/it]


In [125]:
def download_papers(paper_file):
    with open(paper_file, 'r') as file:
        papers_data = json.load(file)
    base_folder = 'papers/'
    for paper in papers_data:
        time.sleep(_CRAWLING_WAIT_TIME)
        year = paper['year']
        pdf_url = paper['url']
        year_folder = os.path.join(base_folder, year)
        os.makedirs(year_folder, exist_ok=True)
        response = requests.get(pdf_url)
        if response.status_code == 200:
            filename = paper['title'].replace(' ', '_')+'.pdf'
            file_path = os.path.join(year_folder, filename)
            with open(file_path, 'wb') as file:
                file.write(response.content)
        

In [97]:
download_papers('dataset.json')

papers/2012/Topology_Constraints_in_Graphical_Models.pdf
papers/2012/Clustering_Aggregation_as_Maximum-Weight_Independent_Set.pdf
papers/2012/FastEx:_Hash_Clustering_with_Exponential_Families.pdf


KeyboardInterrupt: 

In [314]:
import requests
grobid_api_url = "http://localhost:8070/api/processFulltextDocument"
pdf_file_path = "https://proceedings.neurips.cc/paper_files/paper/2014/file/ffeed84c7cb1ae7bf4ec4bd78275bb98-Paper.pdf"


def pdf_2_xml(pdf_file_path):
    # Open and read the PDF file
    files = {'input': requests.get(pdf_file_path).content}

    response = requests.post(grobid_api_url, files=files)
    if response.status_code == 200:
        extracted_data = response.text
    else:
        print(f"Error {response.status_code}: {response.text}")
    return extracted_data


In [315]:
import xml.etree.ElementTree as ET
def extract_email(xml):
    xml_content = xml.replace('<?xml version="1.0" encoding="UTF-8"?>', '')

    # Analyser le contenu XML
   
    root = etree.fromstring(xml_content)
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Utiliser XPath pour trouver les adresses e-mail
    emails = root.xpath('//tei:email/text()', namespaces=namespaces)
    return emails

def extract_author_info(xml):
    xml_content = xml.replace('<?xml version="1.0" encoding="UTF-8"?>', '')
    root = ET.fromstring(xml_content)
    bibl_struct = root.find('.//tei:sourceDesc/tei:biblStruct', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
    authors_info = []
    if bibl_struct is not None:
        for author_element in bibl_struct.findall('.//tei:author', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
            author_info = {}
            pers_name = author_element.find('tei:persName', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            if pers_name is not None:
                author_info['first_name'] = pers_name.findtext('tei:forename[@type="first"]', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                author_info['middle_name'] = pers_name.findtext('tei:forename[@type="middle"]', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                author_info['surname'] = pers_name.findtext('tei:surname', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                author_info['email'] = author_element.findtext('tei:email', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
                # Append author information to the list
                authors_info.append(author_info)
    return authors_info

def extract_ref_from_pdf(pdf_file_path):
    grobid_url = 'http://localhost:8070/api/processReferences'
    # Prepare the files parameter for the API request
    files = {'input': requests.get(pdf_file_path).content}
    response = requests.post(grobid_url, files=files)
    if response.status_code == 200:
        # Grobid renvoie les données au format TEI XML
        grobid_response = response.text
        # Analysez ici le XML pour extraire les informations
        return grobid_response
    else:
        return "Erreur lors de la communication avec Grobid."

In [316]:
refs = extract_ref_from_pdf(pdf_file_path)
def extract_titles_and_authors(xml_content):
    # Analyser le contenu XML
    root = etree.fromstring(xml_content)
    references = []

    # Parcourir chaque référence
    for biblStruct in root.xpath('//tei:biblStruct', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
        # Extraire le titre
        title = biblStruct.xpath('.//tei:title[@level="a" and @type="main"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
        title = title[0] if title else "Unknown"

        # Extraire les auteurs
        authors = []
        for author in biblStruct.xpath('.//tei:author/tei:persName', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
            forenames = author.xpath('.//tei:forename[@type="first"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            middlenames = author.xpath('.//tei:forename[@type="middle"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            surname = author.xpath('.//tei:surname/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            fullname = " ".join(forenames + middlenames + surname)
            authors.append(fullname)

        references.append({'title': title, 'authors': ', '.join(authors)})

    return references

In [326]:
def get_authors_info(pdf_path):
    pdf_xml = pdf_2_xml(pdf_path)
    #emails
    info = extract_author_info(pdf_xml)
    return info

def get_citations(pdf_path):
    xml = extract_ref_from_pdf(pdf_file_path)
    refs = extract_titles_and_authors(xml)
    return refs

In [276]:
info = get_authors_info(pdf_file_path)
print(info)

[{'first_name': 'Neil', 'middle_name': 'M T', 'surname': 'Houlsby', 'email': 'neilhoulsby@google.com'}, {'first_name': 'David', 'middle_name': 'M', 'surname': 'Blei', 'email': 'david.blei@colombia.edu'}]


In [327]:
get_citations(pdf_file_path)

[{'title': 'Stochastic variational inference',
  'authors': 'M D Hoffman, D M Blei, C Wang, J Paisley'},
 {'title': 'Online learning for latent Dirichlet allocation',
  'authors': 'M D Hoffman, D M Blei, F Bach'},
 {'title': 'Unknown',
  'authors': 'J M Hernandez-Lobato, N M T Houlsby, Z Ghahramani'},
 {'title': 'Efficient discovery of overlapping communities in massive networks',
  'authors': 'P K Gopalan, D M Blei'},
 {'title': 'A scalable approach to probabilistic latent space inference of large-scale networks',
  'authors': 'J Yin, Q Ho, E Xing'},
 {'title': 'Unknown', 'authors': 'J Hensman, N Fusi, N D Lawrence'},
 {'title': 'A stochastic approximation method',
  'authors': 'H Robbins, S Monro'},
 {'title': 'An adaptive learning rate for stochastic variational inference',
  'authors': 'R Ranganath, C Wang, D M Blei, E P Xing'},
 {'title': 'Natural gradient works efficiently in learning',
  'authors': 'Shun-Ichi Amari'},
 {'title': 'A new approach to linear filtering and prediction

In [317]:
import sqlite3
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for item in papers_data:
    cursor.execute("INSERT INTO articles (title, year, url, abstract,location) VALUES (?, ?, ?, ?,?)",
                   (item['title'], item['year'], item['url'], item['abstract'],'Papers/'+str(item['year']) + '/'+item['title']+'.pdf'))
conn.commit()
conn.close()

In [319]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('nips.db')  # Replace with the actual path to your SQLite database file

# Query to select all rows from the 'your_table' table
query = 'SELECT * FROM articles'

# Use the connection and query to read data into a DataFrame
df = pd.read_sql_query(query, conn)
conn.close()
df.head()

,id,title,year,url,abstract,location
0,1,Topology Constraints in Graphical Models,2012,https://proceedings.neurips.cc/paper_files/pap...,Graphical models are a very useful tool to des...,Papers/2012/Topology Constraints in Graphical ...
1,2,Clustering Aggregation as Maximum-Weight Indep...,2012,https://proceedings.neurips.cc/paper_files/pap...,We formulate clustering aggregation as a speci...,Papers/2012/Clustering Aggregation as Maximum-...
2,3,FastEx: Hash Clustering with Exponential Families,2012,https://proceedings.neurips.cc/paper_files/pap...,Clustering is a key component in data analysis...,Papers/2012/FastEx: Hash Clustering with Expon...
3,4,The Bethe Partition Function of Log-supermodul...,2012,https://proceedings.neurips.cc/paper_files/pap...,"Sudderth, Wainwright, and Willsky conjectured ...",Papers/2012/The Bethe Partition Function of Lo...
4,5,Selective Labeling via Error Bound Minimization,2012,https://proceedings.neurips.cc/paper_files/pap...,"In many practical machine learning problems, t...",Papers/2012/Selective Labeling via Error Bound...


In [283]:
## Building an author dataset
def get_authors_json(papers_metadata):
    with open(papers_metadata, 'r') as file:
        papers_data = json.load(file)
    author_json = []
    for article in papers_data:
        pdf_link = article['url']
        author_json += get_authors_info(pdf_link)
    with open('authors.json', 'w') as file:
        json.dump(author_json, file, indent=2)        

get_authors_json('test_json.json') #Storing the authors info in a JSON file


In [320]:
#Going from JSON to SQL (Building the author SQL DB)
authors_file = 'authors.json'
with open(authors_file, 'r') as file:
        author_data = json.load(file)
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for item in author_data:
    cursor.execute("INSERT INTO authors (first_name, middle_name, last_name, email) VALUES (?, ?, ?,?)",
                   (item['first_name'], item['middle_name'], item['surname'], item['email']))
conn.commit()
conn.close()

In [336]:
# Connect to the SQLite database
conn = sqlite3.connect('nips.db')  # Replace with the actual path to your SQLite database file

# Query to select all rows from the 'your_table' table
query = 'SELECT * FROM authors'

# Use the connection and query to read data into a DataFrame
df = pd.read_sql_query(query, conn)
conn.close()

In [337]:
df.head(10)

,id,first_name,last_name,middle_name,email
0,1,Marcelo,Fiori,None,mfiori@fing.edu.uy
1,2,Pablo,Musé,None,None
2,3,Guillermo,Sapiro,None,guillermo.sapiro@duke.edu
3,4,Nan,Li,None,nan.li@temple.edu
4,5,Longin,Latecki,Jan,latecki@temple.edu
5,6,Amr,Ahmed,None,amra@google.com
6,7,Sujith,Ravi,None,sravi@google.com
7,8,Shravan,Narayanamurthy,M,shravanmn@gmail.com
8,9,Alexander,Smola,J,None


In [329]:
#Setting up a DB for article_id and author_id
def get_article_id(db_path,article_title):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"SELECT id FROM articles WHERE title = ?", (article_title,))
    result = cursor.fetchone()

    # Check if the article was found
    if result:
        article_id = result[0]
        conn.close()
        return article_id
    else:
        print(f"No article found with the title '{article_title}'.")
        conn.close()
    
        
def get_author_id(db_path,first_name,last_name,email):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    if email :
        cursor.execute("SELECT id FROM authors WHERE first_name = ? AND last_name = ? AND email = ?",
                       (first_name,last_name,email))
    else:
        cursor.execute("SELECT id FROM authors WHERE first_name = ? AND last_name = ?",
                       (first_name,last_name))
    result = cursor.fetchone()
    if result:
        author_id = result[0]
        conn.close()
        return author_id
    else:
        print(f"No author found with the name '{first_name} {last_name}'.")
    conn.close()

with open('test_json.json', 'r') as file:
    article_metadata = json.load(file)
written_by = []
db_path = 'nips.db'
for article in article_metadata:
    #Retrieve the article ID from DB
    article_id = get_article_id(db_path,article['title'])
    #Retrieve authors ID
    #Fetch name and email from URL
    authors = get_authors_info(article['url'])
    for author in authors:
        author_id = get_author_id(db_path,author['first_name'],author['surname'],author['email'])
        written_by.append((article_id,author_id))

In [332]:
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for item in written_by:
    cursor.execute("INSERT INTO written_by (article_id,author_id) VALUES (?, ?)",
                   (item))
conn.commit()
conn.close()

In [335]:
conn = sqlite3.connect('nips.db')  # Replace with the actual path to your SQLite database file
query = 'SELECT * FROM written_by'
df = pd.read_sql_query(query, conn)
conn.close()
df.head()

,article_id,author_id
0,1,1
1,1,2
2,1,3
3,2,4
4,2,5


In [333]:
# Setting up the references DB
with open('test_json.json', 'r') as file:
    article_metadata = json.load(file)
references = []
db_path = 'nips.db'
for article in article_metadata:
    #Retrieve the article ID from DB
    article_id = get_article_id(db_path,article['title'])
    refs = get_citations(article['url'])
    for ref in refs : 
        references.append((article_id,ref['title'],ref['authors']))

In [338]:
conn = sqlite3.connect('nips.db')
cursor = conn.cursor()
for ref in references:
    cursor.execute("INSERT INTO citations (article_id,title,authors) VALUES (?, ?,?)",
                   (ref))
conn.commit()
conn.close()

In [339]:
conn = sqlite3.connect('nips.db')  # Replace with the actual path to your SQLite database file
query = 'SELECT * FROM citations'
df = pd.read_sql_query(query, conn)
conn.close()
df.head()

,article_id,title,authors
0,1,Stochastic variational inference,"M D Hoffman, D M Blei, C Wang, J Paisley"
1,1,Online learning for latent Dirichlet allocation,"M D Hoffman, D M Blei, F Bach"
2,1,Unknown,"J M Hernandez-Lobato, N M T Houlsby, Z Ghahramani"
3,1,Efficient discovery of overlapping communities...,"P K Gopalan, D M Blei"
4,1,A scalable approach to probabilistic latent sp...,"J Yin, Q Ho, E Xing"
